In [1]:
from urllib.request import urlopen

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import re

In [15]:
def is_beer_entry(table_row):
    row_cells=table_row.findAll("td")
    beer_id=get_beer_id(row_cells[0].text)
    return(len(row_cells)==8 and beer_id)

def get_beer_id(cell_value):
    r=re.match("^(\d{1,4})\.$", cell_value)
    if r and len(r.groups())==1:
        beer_id=r.group(1)
        return int(beer_id)
    else:
        return None

In [16]:
def get_all_beers(html_soup):
    beers=[]
    all_rows_in_html_page=html_soup.findAll("tr")
    for table_row in all_rows_in_html_page:
            if is_beer_entry(table_row):
                row_cells=table_row.findAll("td")
                beer_entry={
                    "id":get_beer_id(row_cells[0].text),
                    "name": row_cells[1].text,
                    "brewery_name": row_cells[2].text,
                    "brewery_location": row_cells[3].text,
                    "style": row_cells[4].text,
                    "size": row_cells[5].text,
                    "abv": row_cells[6].text,    
                    "ibu": row_cells[7].text
                }
                beers.append(beer_entry)
    return beers

In [17]:
html=urlopen("http://craftcans.com/db.php?search=all&sort=beerid&ord=desc&view=text")
html_soup=BeautifulSoup(html,'html.parser')
beers_list=get_all_beers(html_soup)

In [28]:
df=pd.DataFrame(beers_list)
df.head()

,abv,brewery_location,brewery_name,ibu,id,name,size,style
0,4.5%,"Minneapolis, MN",NorthGate Brewing,50,2692,Get Together,16 oz.,American IPA
1,4.9%,"Minneapolis, MN",NorthGate Brewing,26,2691,Maggie's Leap,16 oz.,Milk / Sweet Stout
2,4.8%,"Minneapolis, MN",NorthGate Brewing,19,2690,Wall's End,16 oz.,English Brown Ale
3,6.0%,"Minneapolis, MN",NorthGate Brewing,38,2689,Pumpion,16 oz.,Pumpkin Ale
4,6.0%,"Minneapolis, MN",NorthGate Brewing,25,2688,Stronghold,16 oz.,American Porter


In [50]:
brewer=df[["brewery_location","brewery_name"]]
brewer=brewer.drop_duplicates().reset_index(drop=True)
brewer["id"]=brewer.index
brewer.head()

,brewery_location,brewery_name,id
0,"Minneapolis, MN",NorthGate Brewing,0
1,"Louisville, KY",Against the Grain Brewery,1
2,"Framingham, MA",Jack's Abby Craft Lagers,2
3,"San Diego, CA",Mike Hess Brewing Company,3
4,"San Francisco, CA",Fort Point Beer Company,4


In [51]:
beers=pd.merge(df, brewer, 
               left_on=["brewery_name", "brewery_location"],
               right_on=["brewery_name", "brewery_location"],
               sort=True,
               suffixes=('_beer','_brewery'))

In [52]:
beers.head()

,abv,brewery_location,brewery_name,ibu,id_beer,name,size,style,id_brewery
0,5.0%,"Bend, OR",10 Barrel Brewing Company,N/A,1436,Pub Beer,12 oz.,American Pale Lager,408
1,6.6%,"Gary, IN",18th Street Brewery,N/A,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,7.1%,"Gary, IN",18th Street Brewery,N/A,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,9.0%,"Gary, IN",18th Street Brewery,N/A,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,7.5%,"Gary, IN",18th Street Brewery,N/A,2262,Sex and Candy,12 oz.,American IPA,177


In [53]:
beers=beers[["abv", "ibu", "id_beer","name", "size","style",'id_brewery']]
beers_col={"id_beer":"id","id_brewery":'id_brewer'}
beers.rename(inplace=True, columns=beers_col)
beers.head()

,abv,ibu,id,name,size,style,id_brewer
0,5.0%,N/A,1436,Pub Beer,12 oz.,American Pale Lager,408
1,6.6%,N/A,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,7.1%,N/A,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,9.0%,N/A,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,7.5%,N/A,2262,Sex and Candy,12 oz.,American IPA,177


In [54]:
brewer["city"]=[brewer["brewery_location"][i].split(",")[0] for i in range(len(brewer))]

In [55]:
brewer.head()

,brewery_location,brewery_name,id,city
0,"Minneapolis, MN",NorthGate Brewing,0,Minneapolis
1,"Louisville, KY",Against the Grain Brewery,1,Louisville
2,"Framingham, MA",Jack's Abby Craft Lagers,2,Framingham
3,"San Diego, CA",Mike Hess Brewing Company,3,San Diego
4,"San Francisco, CA",Fort Point Beer Company,4,San Francisco


In [56]:
brewer["state"]=[brewer["brewery_location"][i].split(",")[1] for i in range(len(brewer))]

In [57]:
brewer=brewer[['brewery_name',"city","state","id"]]
brewer.rename(inplace=True,columns={"brewery_name":"name"})

In [58]:
brewer.head()

,name,city,state,id
0,NorthGate Brewing,Minneapolis,MN,0
1,Against the Grain Brewery,Louisville,KY,1
2,Jack's Abby Craft Lagers,Framingham,MA,2
3,Mike Hess Brewing Company,San Diego,CA,3
4,Fort Point Beer Company,San Francisco,CA,4


In [59]:
def str_pct_to_float(value):
    value=str(value).strip("%")
    try:
        return float(value)/100
    except ValueError:
        return None
beers["abv"]=beers["abv"].apply(str_pct_to_float)

In [61]:
def str_to_int(val):
    try:
        return int(val)
    except ValueError:
        return None
beers["ibu"]=beers["ibu"].apply(str_to_int)

In [62]:
beers.head()

,abv,ibu,id,name,size,style,id_brewer
0,0.050,NaN,1436,Pub Beer,12 oz.,American Pale Lager,408
1,0.066,NaN,2265,Devil's Cup,12 oz.,American Pale Ale (APA),177
2,0.071,NaN,2264,Rise of the Phoenix,12 oz.,American IPA,177
3,0.090,NaN,2263,Sinister,12 oz.,American Double / Imperial IPA,177
4,0.075,NaN,2262,Sex and Candy,12 oz.,American IPA,177


In [63]:
beers["size"].value_counts()

12 oz.                 1473
16 oz.                  814
12 oz.                   34
24 oz.                   21
19.2 oz.                 14
12 oz                    14
16 oz                    12
16 oz. Alumi-Tek®         9
16 oz. Alumi-Tek®\t       6
32 oz.                    5
8.4 oz.                   1
12 ounce                  1
16.9 oz.                  1
12 oz. Slimline           1
24 oz. "Silo Can"         1
12 & 16 oz.               1
12 OZ.                    1
19.2                      1
Name: size, dtype: int64

In [64]:
def extract_ounces(values):
    stripped=values.strip('oz')
    match=re.match("(\d{1,2}\.*\d*)",stripped)
    if match:
        return float(match.group(0))
    else:
        return None
beers["ounces"]=beers["size"].apply(extract_ounces)
del beers["size"]
beers.head()
    

,abv,ibu,id,name,style,id_brewer,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0
